#### Original Summary:
Notebook to process the raw ATAC data and call peaks (first unfiltered with Homer, then reproducible with IDR). I'll then use the H3K27ac data to call positive and negative peaks. 
I will need some packages found in the deep lift conda env Zeyang had me make... I've added that env to my jupyter notebook, so just need to change the kernel to that one I think.

Adjusted on 1/21/21 to generate peaks for both control and NASH treatment in this notebook!

Output files:
- strain_treat_annot.txt: original peak annotation file 
- strain_treat_filt_annot.txt: filtered peaks with H3K27ac > 16 and "intron" or "Intergenic"
- strain_treat_filt_annot_cut.bed: bed file of ^ with chrY and chrUn peaks removed

#### 1/31 Problem Solving: 
There was very little overlap (even with mergePeaks d=1) between the C57 control and NASH ATAC peaks which is super wrong. Normally we'd see 50-80% overlapping peaks so I'm going to figure out how this happened here. It looks like my raw idr peaks overlap well with Hunter's, so the issue is probably after the step to generate these. 

### PROBLEM WAS d =1 (now should be given)

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import csv
import os

from scipy.stats.stats import pearsonr,spearmanr

from os import listdir
import fnmatch

# Make Genome Browser Hub

Hunter changed these tag directories a bit so skipping this (BUT STILL DOESN'T WORK EITHER)

In [109]:
%%bash
cd /home/h1bennet/strains/data/ATAC/control/
tag_dir=$(ls -d *)
echo $tag_dir
makeMultiWigHub.pl C57Kupffer-ATAC-control2 mm10 -url http://homer.ucsd.edu/hubs/ -webdir /homer_data/www/html/hubs/ -d $tag_dir -force #&>/dev/null &

#normally get a link from this (then put that into the UCSC genome browser)

00_NCoRWT_Kupffer_ATAC_control_young_LN12aM_JSS_150812 00_NCoRWT_Kupffer_ATAC_control_young_NNaF_JSS_150811 01_balbc_Kupffer_ATAC_control_young_balb10A_TDT_JSS_20161013 01_balbc_Kupffer_ATAC_control_young_balb10B_TDT_JSS_20161013 01_balbc_Kupffer_ATAC_control_young_balb10C_TDT_JSS_20161013 02_aj_Kupffer_ATAC_control_young_aj10A_TDT_JSS_20161013 02_aj_Kupffer_ATAC_control_young_aj10B_TDT_JSS_20161013 02_aj_Kupffer_ATAC_control_young_aj10C_TDT_JSS_20161013


Colors that will be used (change with -color or -gradient options):
	Index	Color	Neg. color	Tag Directory
	1	255,150,150	(255,180,180)	00_NCoRWT_Kupffer_ATAC_control_young_LN12aM_JSS_150812
	2	150,150,255	(180,180,255)	00_NCoRWT_Kupffer_ATAC_control_young_NNaF_JSS_150811
	3	150,255,150	(180,255,180)	01_balbc_Kupffer_ATAC_control_young_balb10A_TDT_JSS_20161013
	4	255,200,150	(255,210,180)	01_balbc_Kupffer_ATAC_control_young_balb10B_TDT_JSS_20161013
	5	150,255,220	(180,255,240)	01_balbc_Kupffer_ATAC_control_young_balb10C_TDT_JSS_20161013
	6	200,150,255	(210,180,255)	02_aj_Kupffer_ATAC_control_young_aj10A_TDT_JSS_20161013
	7	200,200,150	(210,210,170)	02_aj_Kupffer_ATAC_control_young_aj10B_TDT_JSS_20161013
	8	150,200,200	(170,210,210)	02_aj_Kupffer_ATAC_control_young_aj10C_TDT_JSS_20161013


	Once finished, you will want to upload the following hub URL:
		http://homer.ucsd.edu/hubs//C57Kupffer-ATAC-control/hub.txt

	If loading to the Wash U Epigenome Browser, use:
		http://homer.ucsd.edu/h

In [110]:
%%bash
cd /home/h1bennet/strains/data/ATAC/AMLN_30week/
tag_dir=$(ls -d *)
echo $tag_dir
makeMultiWigHub.pl C57Kupffer-ATAC-AMLN30wk mm10 -url http://homer.ucsd.edu/hubs/ -webdir /homer_data/www/html/hubs/ -d $tag_dir -force #&>/dev/null &

00_NCoRWT_Kupffer_ATAC_AMLNDiet_30week_LN136C_JSS_TDT_160919 00_NCoRWT_Kupffer_ATAC_AMLNDiet_30week_LN141A_JSS_TDT_160921 01_balbc_Kupffer_ATAC_AMLNDiet_30week_Balb3C_JSS_TDT_160926 01_balbc_Kupffer_ATAC_AMLNDiet_30week_Balb3D_JSS_TDT_160928 02_aj_Kupffer_ATAC_AMLNDiet_30week_AJ3A_JSS_TDT_160926 02_aj_Kupffer_ATAC_AMLNDiet_30week_AJ3B_JSS_TDT_160926


Colors that will be used (change with -color or -gradient options):
	Index	Color	Neg. color	Tag Directory
	1	255,150,150	(255,180,180)	00_NCoRWT_Kupffer_ATAC_AMLNDiet_30week_LN136C_JSS_TDT_160919
	2	150,150,255	(180,180,255)	00_NCoRWT_Kupffer_ATAC_AMLNDiet_30week_LN141A_JSS_TDT_160921
	3	150,255,150	(180,255,180)	01_balbc_Kupffer_ATAC_AMLNDiet_30week_Balb3C_JSS_TDT_160926
	4	255,200,150	(255,210,180)	01_balbc_Kupffer_ATAC_AMLNDiet_30week_Balb3D_JSS_TDT_160928
	5	150,255,220	(180,255,240)	02_aj_Kupffer_ATAC_AMLNDiet_30week_AJ3A_JSS_TDT_160926
	6	200,150,255	(210,180,255)	02_aj_Kupffer_ATAC_AMLNDiet_30week_AJ3B_JSS_TDT_160926


	Once finished, you will want to upload the following hub URL:
		http://homer.ucsd.edu/hubs//C57Kupffer-ATAC-AMLN30wk/hub.txt

	If loading to the Wash U Epigenome Browser, use:
		http://homer.ucsd.edu/hubs//C57Kupffer-ATAC-AMLN30wk/washU.hub.txt

print() on closed filehandle HUBLIST at /bioinformatics/homer/bin/makeMultiWigHub.pl line 388, <IN> line 28.

	Visualiz

# Call Peaks

## Step 1: Unfiltered peaks from HOMER

### Control

File names:
- 00_NCoRWT_Kupffer_ATAC_control_young_LN12aM_JSS_150812
- 00_NCoRWT_Kupffer_ATAC_control_young_NNaF_JSS_150811

In [16]:
%%bash
cd /home/hmummey/data/ATAC/raw_peaks/Control
for data in ./*NCoRWT*; do
    strain="$(cut -d '_' -f2 <<<"$data")"
    treatment="$(cut -d '_' -f5 <<<"$data")"    
    name="$(cut -d '/' -f2 <<<"$data")_round2"
    echo $name
    
    findPeaks $data -style factor -L 0 -C 0 -fdr 0.9 -o /home/hmummey/data/ATAC/init_peak_calling/${name}.txt -size 200 &>/dev/null &
done

00_NCoRWT_Kupffer_ATAC_control_young_LN12aM_JSS_150812_round2
00_NCoRWT_Kupffer_ATAC_control_young_NNaF_JSS_150811_round2


In [8]:
tag_diretory_array = np.array(fnmatch.filter(listdir('/home/hmummey/data/ATAC/raw_peaks/Control'), '*NCoRWT*'))
tag_diretory_array.sort()
tag_diretory_array

array(['00_NCoRWT_Kupffer_ATAC_control_young_LN12aM_JSS_150812',
       '00_NCoRWT_Kupffer_ATAC_control_young_NNaF_JSS_150811'],
      dtype='<U54')

### AMLN

In [11]:
%%bash
cd /home/hmummey/data/ATAC/raw_peaks/AMLN_30week
for data in ./*NCoRWT*; do
    strain="$(cut -d '_' -f2 <<<"$data")"
    treatment="$(cut -d '_' -f5 <<<"$data")"    
    name="$(cut -d '/' -f2 <<<"$data")_round2"
    echo $name
    findPeaks $data -style factor -L 0 -C 0 -fdr 0.9 -o /home/hmummey/data/ATAC/init_peak_calling/${name}.txt -size 200 &>/dev/null &
done

00_NCoRWT_Kupffer_ATAC_AMLNDiet_30week_LN136C_JSS_TDT_160919_round2
00_NCoRWT_Kupffer_ATAC_AMLNDiet_30week_LN141A_JSS_TDT_160921_round2


In [12]:
tag_diretory_array2 = np.array(fnmatch.filter(listdir('/home/hmummey/data/ATAC/raw_peaks/AMLN_30week'), '*NCoRWT*'))
tag_diretory_array2.sort()
tag_diretory_array2

array(['00_NCoRWT_Kupffer_ATAC_AMLNDiet_30week_LN136C_JSS_TDT_160919',
       '00_NCoRWT_Kupffer_ATAC_AMLNDiet_30week_LN141A_JSS_TDT_160921'],
      dtype='<U60')

#### Creating a file with all chip_names (control and AMLN)

In [13]:
chip_names = []
for i in tag_diretory_array:
    strain = i.split('_')[1]
    treat = i.split('_')[4]
    chip_names.append('_'.join([strain, treat]))
    
for i in tag_diretory_array2:
    strain = i.split('_')[1]
    treat = i.split('_')[4]
    chip_names.append('_'.join([strain, treat]))
    
chip_names = np.unique(np.array(chip_names))
chip_names

array(['NCoRWT_AMLNDiet', 'NCoRWT_control'], dtype='<U15')

In [14]:
chip_file = '/home/hmummey/data/ATAC/chip_names_list_round2.txt' #exactly the same as prev file
with open(chip_file, 'w') as f:
    for c in chip_names:
        f.write(c + '\n')

## Step 2: Reproducible peaks from IDR

In [27]:
%%bash

cd /home/hmummey/data/ATAC/init_peak_calling/
while IFS='' read -r IP
do
    strain="$(cut -d '_' -f1 <<<"$IP")"
    treatment="$(cut -d '_' -f2 <<<"$IP")"
    round="round2"
    #name="$(cut -d '/' -f2 <<<"$data" | cut -d '.' -f1)"

    echo $IP
    echo ./*${strain}*${treatment}*${round}*
    #echo *${strain}*${treatment}*
    echo 

    #mkdir ../IDR/${IP}
    
    #python3 /home/zes017/run_idr_homerPeaks.py -threshold 0.05 ./*${strain}*${treatment}*${round}* ../IDR/${IP}*${round} #&>/dev/null &
done < '/home/hmummey/data/ATAC/chip_names_list_round2.txt'

NCoRWT_AMLNDiet
./00_NCoRWT_Kupffer_ATAC_AMLNDiet_30week_LN136C_JSS_TDT_160919_round2.txt ./00_NCoRWT_Kupffer_ATAC_AMLNDiet_30week_LN141A_JSS_TDT_160921_round2.txt

NCoRWT_control
./00_NCoRWT_Kupffer_ATAC_control_young_LN12aM_JSS_150812_round2.txt ./00_NCoRWT_Kupffer_ATAC_control_young_NNaF_JSS_150811_round2.txt



# Count Tags of Histone Marks

In [36]:
%%bash

#merged file combines peak files from all the strains (for one condition)
#alt: look at induced peaks by merging from one strain (both conditions)
#HERE WE AREN'T USING MERGED FILES, JUST USE THE PREVIOUSLY MADE .TSV FILES

for peak_file in /home/hmummey/data/ATAC/annotated_peak_calling/*_idr.tsv; do
    echo $peak_file

    #strain="$(echo $peak_file | cut -d '/' -f7 | cut -d '_' -f2)"
    strain="C57"
    treat="$(echo $peak_file | cut -d '/' -f7 | cut -d '_' -f5)"
    
    #treat (AMLNDiet) doesn't match his dir name: /home/h1bennet/strains/data/H3K27Ac/AMLN_30week
    #but it does match the file names: 00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579A_180423
    #so modifying this a bit (cut down and then use * later)
    treat2="$(echo $treat | cut -b 1-4)"

    echo $strain
    echo $treat
    echo $treat2
    
    files=''
    for h in 'H3K27Ac'; do
        files+="$(echo /home/h1bennet/strains/data/$h/$treat2*/*${strain}*${h}*)"

        files+=' '
    done
    echo $files
    echo
    annotatePeaks.pl $peak_file mm10 -size -500,500 -norm 1e7 -d $files > /home/hmummey/data/ATAC/annotated_peak_calling/${strain}_${treat}_round2_annot.txt #2>/dev/null &
done

/home/hmummey/data/ATAC/annotated_peak_calling/00_NCoRWT_Kupffer_ATAC_AMLNDiet_30week_LN136C_JSS_TDT_160919_round2_00_NCoRWT_Kupffer_ATAC_AMLNDiet_30week_LN141A_JSS_TDT_160921_round2_idr.tsv
C57
AMLNDiet
AMLN
/home/h1bennet/strains/data/H3K27Ac/AMLN_30week/00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579A_180423 /home/h1bennet/strains/data/H3K27Ac/AMLN_30week/00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579C_180423

/home/hmummey/data/ATAC/annotated_peak_calling/00_NCoRWT_Kupffer_ATAC_control_young_LN12aM_JSS_150812_round2_00_NCoRWT_Kupffer_ATAC_control_young_NNaF_JSS_150811_round2_idr.tsv
C57
control
cont
/home/h1bennet/strains/data/H3K27Ac/control/00_C57_Kupffer_H3K27Ac_control_young_C571A_170915 /home/h1bennet/strains/data/H3K27Ac/control/00_C57_Kupffer_H3K27Ac_control_young_C572A_180423




	Peak file = /home/hmummey/data/ATAC/annotated_peak_calling/00_NCoRWT_Kupffer_ATAC_AMLNDiet_30week_LN136C_JSS_TDT_160919_round2_00_NCoRWT_Kupffer_ATAC_AMLNDiet_30week_LN141A_JSS_TDT_160921_round2_idr.tsv
	Genome = mm10
	Organism = mouse
	Peak Region set to 1000
	Will normalize tag counts to 1e7 per experiment
	Tag Directories:
		/home/h1bennet/strains/data/H3K27Ac/AMLN_30week/00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579A_180423
		/home/h1bennet/strains/data/H3K27Ac/AMLN_30week/00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579C_180423
	Peak/BED file conversion summary:
		BED/Header formatted lines: 0
		peakfile formatted lines: 46223
		Duplicated Peak IDs: 0

	Peak File Statistics:
		Total Peaks: 46223
		Redundant Peak IDs: 0
		Peaks lacking information: 0 (need at least 5 columns per peak)
		Peaks with misformatted coordinates: 0 (should be integer)
		Peaks with misformatted strand: 0 (should be either +/- or 0/1)

	Peak file looks good!

	Resizing peaks...
	Reading Positions...
	---------

### Control

In [37]:
strain = 'C57_control_round2'
path = '/home/hmummey/data/ATAC/annotated_peak_calling/'
annot_file = path + strain + '_annot.txt'
annot_df = pd.read_csv(annot_file, sep='\t')

In [38]:
print(len(annot_df))
annot_df.head()

48664


,"PeakID (cmd=annotatePeaks.pl /home/hmummey/data/ATAC/annotated_peak_calling/00_NCoRWT_Kupffer_ATAC_control_young_LN12aM_JSS_150812_round2_00_NCoRWT_Kupffer_ATAC_control_young_NNaF_JSS_150811_round2_idr.tsv mm10 -size -500,500 -norm 1e7 -d /home/h1bennet/strains/data/H3K27Ac/control/00_C57_Kupffer_H3K27Ac_control_young_C571A_170915 /home/h1bennet/strains/data/H3K27Ac/control/00_C57_Kupffer_H3K27Ac_control_young_C572A_180423)",Chr,Start,End,Strand,Peak Score,Focus Ratio/Region Size,Annotation,Detailed Annotation,Distance to TSS,...,Entrez ID,Nearest Unigene,Nearest Refseq,Nearest Ensembl,Gene Name,Gene Alias,Gene Description,Gene Type,"/home/h1bennet/strains/data/H3K27Ac/control/00_C57_Kupffer_H3K27Ac_control_young_C571A_170915 Tag Count in 1000 bp (9468035.0 Total, normalization factor = 1.06, effective total = 1e7)","/home/h1bennet/strains/data/H3K27Ac/control/00_C57_Kupffer_H3K27Ac_control_young_C572A_180423 Tag Count in 1000 bp (22615879.0 Total, normalization factor = 0.44, effective total = 1e7)"
0,65_35545,chr8,71633967,71634967,+,1000,28.0,"intron (NM_001029873, intron 39 of 41)","intron (NM_001029873, intron 39 of 41)",-3361.0,...,102466799.0,NaN,NR_106035,ENSMUSG00000098976,Mir6769b,Gm27546|mmu-mir-6769b,microRNA 6769b,ncRNA,6.34,3.54
1,65_22643,chr5,113991974,113992974,+,1000,43.0,"intron (NM_198109, intron 1 of 14)",L1MB8|LINE|L1,1283.0,...,231637.0,Mm.389682,NM_198109,ENSMUSG00000042121,Ssh1,AW551225|Gm1394|Gm1395|SSH-1|SSH-1L,slingshot protein phosphatase 1,protein-coding,67.60,63.23
2,65_14830,chr9,57765476,57766476,+,1000,59.0,promoter-TSS (NM_007713),promoter-TSS (NM_007713),-116.0,...,102414.0,Mm.25720,NM_007713,ENSMUSG00000032316,Clk3,AI256811,CDC-like kinase 3,protein-coding,260.88,279.01
3,65_28521,chr7,97154713,97155713,+,1000,35.0,"intron (NM_001162477, intron 1 of 9)","intron (NM_001162477, intron 1 of 9)",73462.0,...,14389.0,Mm.42033,NM_010248,ENSMUSG00000004508,Gab2,AI463667|D130058I17Rik|p97,growth factor receptor bound protein 2-associa...,protein-coding,46.47,48.20
4,65_16538,chr10,95501369,95502369,+,1000,54.0,promoter-TSS (NM_001359477),promoter-TSS (NM_001359477),80.0,...,67270.0,Mm.30008,NM_026065,ENSMUSG00000062981,Mrpl42,2700009F22Rik|2900055D03Rik|D10Ertd322e|HSPC20...,mitochondrial ribosomal protein L42,protein-coding,106.67,99.05


In [51]:
histone_tags1 = annot_df.iloc[:,-2:]
histone_tags1.columns = [i.split(' Tag')[0].split('/')[-1] for i in histone_tags1.columns.values]
histone_tags2 = np.log2(histone_tags1+1) #convert to log2 scale
histone_tags2.head()

,00_C57_Kupffer_H3K27Ac_control_young_C571A_170915,00_C57_Kupffer_H3K27Ac_control_young_C572A_180423
0,2.875780,2.182692
1,6.100137,6.005175
2,8.032762,8.129335
3,5.568944,5.620586
4,6.750473,6.644577


In [52]:
histone_tags2.describe()

,00_C57_Kupffer_H3K27Ac_control_young_C571A_170915,00_C57_Kupffer_H3K27Ac_control_young_C572A_180423
count,48664.000000,48664.000000
mean,5.149920,4.984465
std,1.923150,2.074316
min,0.000000,0.000000
25%,3.657640,3.363171
50%,5.324811,5.131343
75%,6.707497,6.676098
max,15.399944,14.987281


### AMLN Diet

In [56]:
strain2 = 'C57_AMLNDiet_round2'
path = '/home/hmummey/data/ATAC/annotated_peak_calling/'
annot_file2 = path + strain2 + '_annot.txt'
annot_df2 = pd.read_csv(annot_file2, sep='\t')

print(len(annot_df2))
annot_df2.head()

46223


,"PeakID (cmd=annotatePeaks.pl /home/hmummey/data/ATAC/annotated_peak_calling/00_NCoRWT_Kupffer_ATAC_AMLNDiet_30week_LN136C_JSS_TDT_160919_round2_00_NCoRWT_Kupffer_ATAC_AMLNDiet_30week_LN141A_JSS_TDT_160921_round2_idr.tsv mm10 -size -500,500 -norm 1e7 -d /home/h1bennet/strains/data/H3K27Ac/AMLN_30week/00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579A_180423 /home/h1bennet/strains/data/H3K27Ac/AMLN_30week/00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579C_180423)",Chr,Start,End,Strand,Peak Score,Focus Ratio/Region Size,Annotation,Detailed Annotation,Distance to TSS,...,Entrez ID,Nearest Unigene,Nearest Refseq,Nearest Ensembl,Gene Name,Gene Alias,Gene Description,Gene Type,"/home/h1bennet/strains/data/H3K27Ac/AMLN_30week/00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579A_180423 Tag Count in 1000 bp (18749728.0 Total, normalization factor = 0.53, effective total = 1e7)","/home/h1bennet/strains/data/H3K27Ac/AMLN_30week/00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579C_180423 Tag Count in 1000 bp (19231109.0 Total, normalization factor = 0.52, effective total = 1e7)"
0,52_6108,chr19,6869077,6870077,+,1000,88.0,Intergenic,RLTR19-int|LTR|ERVK,-11366.0,...,78317.0,Mm.329596,NM_198008,ENSMUSG00000047810,Ccdc88b,2610041P08Rik|Ccdc88,coiled-coil domain containing 88B,protein-coding,102.40,111.28
1,52_27549,chr7,128136635,128137635,+,1000,32.0,"intron (NM_021334, intron 14 of 29)","intron (NM_021334, intron 14 of 29)",7567.0,...,16411.0,Mm.22378,NM_021334,ENSMUSG00000030789,Itgax,AI449405|Cd11c|Cr4|N418,integrin alpha X,protein-coding,51.73,24.96
2,52_532,chr11,20200837,20201837,+,1000,178.0,promoter-TSS (NM_008996),promoter-TSS (NM_008996),-75.0,...,19324.0,Mm.271944,NM_008996,ENSMUSG00000020149,Rab1a,Gtbp|Rab-1|Rab1|Ypt1|mKIAA3012,"RAB1A, member RAS oncogene family",protein-coding,450.67,384.79
3,52_21363,chr9,64736183,64737183,+,1000,40.0,"intron (NM_017382, intron 1 of 4)","intron (NM_017382, intron 1 of 4)",1073.0,...,53869.0,Mm.1387,NM_017382,ENSMUSG00000004771,Rab11a,-,"RAB11A, member RAS oncogene family",protein-coding,139.74,110.76
4,52_22519,chr4,155561699,155562699,+,1000,38.0,promoter-TSS (NM_138671),promoter-TSS (NM_138671),-182.0,...,192185.0,Mm.28347,NM_138671,ENSMUSG00000029063,Nadk,4432404C02Rik|BC004012,NAD kinase,protein-coding,135.47,114.40


In [57]:
histone_tags12 = annot_df2.iloc[:,-2:]
histone_tags12.columns = [i.split(' Tag')[0].split('/')[-1] for i in histone_tags12.columns.values]
histone_tags22 = np.log2(histone_tags12+1) #convert to log2 scale
histone_tags22.head()

,00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579A_180423,00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579C_180423
0,6.692092,6.810957
1,5.720552,4.698218
2,8.819125,8.591672
3,7.136889,6.804260
4,7.092440,6.850499


In [58]:
histone_tags22.describe()

,00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579A_180423,00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579C_180423
count,46223.000000,46223.000000
mean,4.891646,4.787522
std,2.123845,2.046592
min,0.000000,0.000000
25%,3.252476,3.137504
50%,5.067381,4.912650
75%,6.592008,6.413459
max,15.693303,15.991561


# Identify active enhancers

1) By setting an average threshold on H3K27ac tags > 16 (average 2 replicates, cutoff 16 on the actual tag counts, or 4 on log2 scale)

2) HOMER annotations to be at intronic or intergenic regions

### Control

In [59]:
histone_tags2.columns = ["C57_control_170915", "C57_control_180423"]
z = [((row.C57_control_170915 + row.C57_control_180423)/2) for index, row in histone_tags2.iterrows() ]
print(z[:5])
histone_tags2['C57_control_avg'] = z
histone_tags2.head()

[2.5292361802923393, 6.052656031308695, 8.081048307330793, 5.594765277553292, 6.697524937890444]


,C57_control_170915,C57_control_180423,C57_control_avg
0,2.875780,2.182692,2.529236
1,6.100137,6.005175,6.052656
2,8.032762,8.129335,8.081048
3,5.568944,5.620586,5.594765
4,6.750473,6.644577,6.697525


In [60]:
#glancing at annotations
print((annot_df.iloc[:,7])[:10])

0    intron (NM_001029873, intron 39 of 41)
1        intron (NM_198109, intron 1 of 14)
2                  promoter-TSS (NM_007713)
3      intron (NM_001162477, intron 1 of 9)
4               promoter-TSS (NM_001359477)
5     intron (NM_001040111, intron 1 of 33)
6               promoter-TSS (NM_001083334)
7                                Intergenic
8       intron (NM_011075, intron 26 of 27)
9                                Intergenic
Name: Annotation, dtype: object


In [61]:
#just gonna loop through all rows (for both dfs) and record rows to save as those with:
#histone > 16 (average 2 replicates, cutoff 16 on the actual tag counts, or 4 on log2 scale)
#Annotation containing either: intron, Intergenic

save_rows = []
for row in np.arange(len(annot_df)):
    #grab full annotation
    full_annot = str(annot_df.iloc[row,7])
    if ("intron" in full_annot or "Intergenic" in full_annot) and (histone_tags2.iloc[row,2] > 4):
        save_rows.append(row)

In [62]:
print(len(save_rows))
print(save_rows[:10]) #looks good!

22282
[1, 3, 5, 7, 10, 13, 14, 15, 16, 17]


In [63]:
filt_annot = annot_df.iloc[save_rows,:]
print(len(filt_annot))
filt_annot.head()

22282


,"PeakID (cmd=annotatePeaks.pl /home/hmummey/data/ATAC/annotated_peak_calling/00_NCoRWT_Kupffer_ATAC_control_young_LN12aM_JSS_150812_round2_00_NCoRWT_Kupffer_ATAC_control_young_NNaF_JSS_150811_round2_idr.tsv mm10 -size -500,500 -norm 1e7 -d /home/h1bennet/strains/data/H3K27Ac/control/00_C57_Kupffer_H3K27Ac_control_young_C571A_170915 /home/h1bennet/strains/data/H3K27Ac/control/00_C57_Kupffer_H3K27Ac_control_young_C572A_180423)",Chr,Start,End,Strand,Peak Score,Focus Ratio/Region Size,Annotation,Detailed Annotation,Distance to TSS,...,Entrez ID,Nearest Unigene,Nearest Refseq,Nearest Ensembl,Gene Name,Gene Alias,Gene Description,Gene Type,"/home/h1bennet/strains/data/H3K27Ac/control/00_C57_Kupffer_H3K27Ac_control_young_C571A_170915 Tag Count in 1000 bp (9468035.0 Total, normalization factor = 1.06, effective total = 1e7)","/home/h1bennet/strains/data/H3K27Ac/control/00_C57_Kupffer_H3K27Ac_control_young_C572A_180423 Tag Count in 1000 bp (22615879.0 Total, normalization factor = 0.44, effective total = 1e7)"
1,65_22643,chr5,113991974,113992974,+,1000,43.0,"intron (NM_198109, intron 1 of 14)",L1MB8|LINE|L1,1283.0,...,231637.0,Mm.389682,NM_198109,ENSMUSG00000042121,Ssh1,AW551225|Gm1394|Gm1395|SSH-1|SSH-1L,slingshot protein phosphatase 1,protein-coding,67.60,63.23
3,65_28521,chr7,97154713,97155713,+,1000,35.0,"intron (NM_001162477, intron 1 of 9)","intron (NM_001162477, intron 1 of 9)",73462.0,...,14389.0,Mm.42033,NM_010248,ENSMUSG00000004508,Gab2,AI463667|D130058I17Rik|p97,growth factor receptor bound protein 2-associa...,protein-coding,46.47,48.20
5,65_34295,chr7,101360702,101361702,+,1000,29.0,"intron (NM_001040111, intron 1 of 33)","intron (NM_001040111, intron 1 of 33)",13133.0,...,69710.0,Mm.277687,NM_027180,ENSMUSG00000032812,Arap1,2410002L19Rik|Centd2|mKIAA0782,"ArfGAP with RhoGAP domain, ankyrin repeat and ...",protein-coding,191.17,166.70
7,65_32720,chr15,97949323,97950323,+,1000,30.0,Intergenic,Intergenic,-14406.0,...,380967.0,Mm.196630,NM_201359,ENSMUSG00000052369,Tmem106c,AI046681|BC046621|D15Ertd405e,transmembrane protein 106C,protein-coding,85.55,63.23
10,65_8821,chr2,164868133,164869133,+,1000,79.0,Intergenic,MTC|LTR|ERVL-MaLR,-10735.0,...,228866.0,Mm.241134,NM_146129,ENSMUSG00000039849,Pcif1,2310022K11Rik|C20orf67|F730014I05Rik,PDX1 C-terminal inhibiting factor 1,protein-coding,66.54,88.88


In [64]:
#export this just in case
out_file = path + strain + "_filt_annot.txt"
filt_annot.to_csv(out_file, index=False)

### AMLN

In [65]:
histone_tags22.columns = ["C57_AMLN1", "C57_control_AMLN2"]
z2 = [((row.C57_AMLN1 + row.C57_control_AMLN2)/2) for index, row in histone_tags22.iterrows() ]
print(z2[:5])
histone_tags22['C57_AMLN_avg'] = z2
histone_tags22.head()

[6.751524767082686, 5.209385284914761, 8.705398610229484, 6.970574361768902, 6.971469721998103]


,C57_AMLN1,C57_control_AMLN2,C57_AMLN_avg
0,6.692092,6.810957,6.751525
1,5.720552,4.698218,5.209385
2,8.819125,8.591672,8.705399
3,7.136889,6.804260,6.970574
4,7.092440,6.850499,6.971470


In [66]:
#glancing at annotations
print((annot_df2.iloc[:,7])[:10])

0                             Intergenic
1    intron (NM_021334, intron 14 of 29)
2               promoter-TSS (NM_008996)
3      intron (NM_017382, intron 1 of 4)
4               promoter-TSS (NM_138671)
5               promoter-TSS (NM_175542)
6     intron (NM_031376, intron 2 of 16)
7               promoter-TSS (NM_026050)
8      intron (NM_007651, intron 1 of 7)
9            promoter-TSS (NM_001048267)
Name: Annotation, dtype: object


In [67]:
#just gonna loop through all rows (for both dfs) and record rows to save as those with:
#histone > 16 (average 2 replicates, cutoff 16 on the actual tag counts, or 4 on log2 scale)
#Annotation containing either: intron, Intergenic

save_rows2 = []
for row in np.arange(len(annot_df2)):
    #grab full annotation
    full_annot2 = str(annot_df2.iloc[row,7])
    if ("intron" in full_annot2 or "Intergenic" in full_annot2) and (histone_tags22.iloc[row,2] > 4):
        save_rows2.append(row)
        
print(len(save_rows2))
print(save_rows2[:10]) #looks good!

19161
[0, 1, 3, 6, 8, 11, 12, 13, 14, 15]


In [68]:
filt_annot2 = annot_df2.iloc[save_rows2,:]
print(len(filt_annot2))
filt_annot2.head()

19161


,"PeakID (cmd=annotatePeaks.pl /home/hmummey/data/ATAC/annotated_peak_calling/00_NCoRWT_Kupffer_ATAC_AMLNDiet_30week_LN136C_JSS_TDT_160919_round2_00_NCoRWT_Kupffer_ATAC_AMLNDiet_30week_LN141A_JSS_TDT_160921_round2_idr.tsv mm10 -size -500,500 -norm 1e7 -d /home/h1bennet/strains/data/H3K27Ac/AMLN_30week/00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579A_180423 /home/h1bennet/strains/data/H3K27Ac/AMLN_30week/00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579C_180423)",Chr,Start,End,Strand,Peak Score,Focus Ratio/Region Size,Annotation,Detailed Annotation,Distance to TSS,...,Entrez ID,Nearest Unigene,Nearest Refseq,Nearest Ensembl,Gene Name,Gene Alias,Gene Description,Gene Type,"/home/h1bennet/strains/data/H3K27Ac/AMLN_30week/00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579A_180423 Tag Count in 1000 bp (18749728.0 Total, normalization factor = 0.53, effective total = 1e7)","/home/h1bennet/strains/data/H3K27Ac/AMLN_30week/00_C57_Kupffer_H3K27Ac_AMLNDiet_20weeks_C579C_180423 Tag Count in 1000 bp (19231109.0 Total, normalization factor = 0.52, effective total = 1e7)"
0,52_6108,chr19,6869077,6870077,+,1000,88.0,Intergenic,RLTR19-int|LTR|ERVK,-11366.0,...,78317.0,Mm.329596,NM_198008,ENSMUSG00000047810,Ccdc88b,2610041P08Rik|Ccdc88,coiled-coil domain containing 88B,protein-coding,102.40,111.28
1,52_27549,chr7,128136635,128137635,+,1000,32.0,"intron (NM_021334, intron 14 of 29)","intron (NM_021334, intron 14 of 29)",7567.0,...,16411.0,Mm.22378,NM_021334,ENSMUSG00000030789,Itgax,AI449405|Cd11c|Cr4|N418,integrin alpha X,protein-coding,51.73,24.96
3,52_21363,chr9,64736183,64737183,+,1000,40.0,"intron (NM_017382, intron 1 of 4)","intron (NM_017382, intron 1 of 4)",1073.0,...,53869.0,Mm.1387,NM_017382,ENSMUSG00000004771,Rab11a,-,"RAB11A, member RAS oncogene family",protein-coding,139.74,110.76
6,52_28164,chr19,41345112,41346112,+,1000,31.0,"intron (NM_031376, intron 2 of 16)","intron (NM_031376, intron 2 of 16)",39458.0,...,83490.0,Mm.222266,NM_031376,ENSMUSG00000025017,Pik3ap1,1810044J04Rik|BCAP,phosphoinositide-3-kinase adaptor protein 1,protein-coding,293.34,274.56
8,52_17359,chr3,106784764,106785764,+,1000,47.0,"intron (NM_007651, intron 1 of 7)","intron (NM_007651, intron 1 of 7)",4885.0,...,12508.0,Mm.316861,NM_007651,ENSMUSG00000040747,Cd53,AI323659|Ox-44|Tspan25,CD53 antigen,protein-coding,141.34,137.80


In [69]:
#export this just in case
out_file2 = path + strain2 + "_filt_annot.txt"
filt_annot.to_csv(out_file2, index=False)

# Convert peak file to BED

Bed file format requires:
- chrom - name of the chromosome or scaffold. Any valid seq_region_name can be used, and chromosome names can be given with or without the 'chr' prefix.
- chromStart - Start position of the feature in standard chromosomal coordinates (i.e. first base is 0).
- chromEnd - End position of the feature in standard chromosomal coordinates

Optional columns: name, score, strand, thickStart, thickEnd, itemRgb, blockCount, blockSizes, blockStarts

from: https://m.ensembl.org/info/website/upload/bed.html

### Control

In [71]:
#just going to use the required cols (and maybe strand and score? why not, I guess)
save_cols = [1,2,3,4,5]
bed_df = filt_annot.iloc[:,save_cols]
print(len(bed_df))
bed_df.head()

22282


,Chr,Start,End,Strand,Peak Score
1,chr5,113991974,113992974,+,1000
3,chr7,97154713,97155713,+,1000
5,chr7,101360702,101361702,+,1000
7,chr15,97949323,97950323,+,1000
10,chr2,164868133,164869133,+,1000


In [72]:
chr_list = list(set(bed_df.iloc[:,0]))
chr_list.sort()
print(chr_list)

['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr2', 'chr3', 'chr4', 'chr4_GL456216_random', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chrUn_JH584304', 'chrX', 'chrY']


Remove all rows that are chrY (makes the script to create the background, error) or chrUn

In [74]:
#also might as well try removing rows with Un chrs (will do this manually and see if helps)
keep_rows12 = []
for i in range(len(bed_df)):
    chr_str = bed_df.iloc[i,0]
    if ("chrY" in chr_str) or ("Un" in chr_str):
        pass
    else:
        keep_rows12.append(i)

print(len(keep_rows12))
print(keep_rows12[:20])

22224
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [75]:
bed_df_cut12 = bed_df.iloc[keep_rows12,:]
print(len(bed_df_cut12))
bed_df_cut12.head()

22224


,Chr,Start,End,Strand,Peak Score
1,chr5,113991974,113992974,+,1000
3,chr7,97154713,97155713,+,1000
5,chr7,101360702,101361702,+,1000
7,chr15,97949323,97950323,+,1000
10,chr2,164868133,164869133,+,1000


In [77]:
out3 = path + strain + "_filt_annot_fin.bed"
bed_df_cut12.to_csv(out3, index=False, sep='\t')

### AMLN

In [78]:
#just going to use the required cols (and maybe strand and score? why not, I guess)

save_cols2 = [1,2,3,4,5]
bed_df2 = filt_annot2.iloc[:,save_cols2]
print(len(bed_df2))
bed_df2.head()

19161


,Chr,Start,End,Strand,Peak Score
0,chr19,6869077,6870077,+,1000
1,chr7,128136635,128137635,+,1000
3,chr9,64736183,64737183,+,1000
6,chr19,41345112,41346112,+,1000
8,chr3,106784764,106785764,+,1000


In [79]:
chr_list = list(set(bed_df2.iloc[:,0]))
chr_list.sort()
print(chr_list)

['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr2', 'chr3', 'chr4', 'chr4_GL456216_random', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chrUn_JH584304', 'chrX', 'chrY']


Remove all rows that are chrY (makes the script to create the background, error) or chrUn

In [80]:
#also might as well try removing rows with Un chrs (will do this manually and see if helps)
keep_rows22 = []
for i in range(len(bed_df2)):
    chr_str = bed_df2.iloc[i,0]
    if ("chrY" in chr_str) or ("Un" in chr_str):
        pass
    else:
        keep_rows22.append(i)
        
print(len(keep_rows22))
print(keep_rows22[:20])

19114
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [81]:
bed_df_cut22 = bed_df2.iloc[keep_rows22,:]
print(len(bed_df_cut22))
bed_df_cut22.head()

19114


,Chr,Start,End,Strand,Peak Score
0,chr19,6869077,6870077,+,1000
1,chr7,128136635,128137635,+,1000
3,chr9,64736183,64737183,+,1000
6,chr19,41345112,41346112,+,1000
8,chr3,106784764,106785764,+,1000


In [82]:
out4 = path + strain2 + "_filt_annot_fin.bed"
bed_df_cut22.to_csv(out4, index=False, sep='\t')

# Generate random backgrounds

Bed file is the positives we've just chosen (only matching GC content). Make sure to remove the top row (col names) in terminal with sed '1d' before running the script!

### Control

In [85]:
%%bash
bed_file="/home/hmummey/data/ATAC/annotated_peak_calling/C57_control_round2_filt_annot_fin.bed" #+ peaks, no chrY, no chrUn
output=$(echo $bed_file | sed 's/.bed/_bg/g')
echo $output
python /home/zes017/Spacing/Codes/generate_background_coordinates.py $bed_file $output

/home/hmummey/data/ATAC/annotated_peak_calling/C57_control_round2_filt_annot_fin_bg
reading genome mm10
fasta file does not exist: chr20
fasta file does not exist: chr21
fasta file does not exist: chr22
done reading genome
0 0
target GC: 0.37210436347276776 background GC: 0.3745772895570298 target length: 1000 numTargetPositions 2223 backgroundPositions 2223
0 0
target GC: 0.406293429342916 background GC: 0.4064176147884336 target length: 1000 numTargetPositions 2222 backgroundPositions 2222
0 0
target GC: 0.4242187218721681 background GC: 0.4224825579460853 target length: 1000 numTargetPositions 2222 backgroundPositions 2222
0 0
target GC: 0.4386300630062809 background GC: 0.435475865268825 target length: 1000 numTargetPositions 2222 backgroundPositions 2222
0 0
target GC: 0.4519702970296826 background GC: 0.4483882454179282 target length: 1000 numTargetPositions 2222 backgroundPositions 2222
0 0
target GC: 0.46530198019799884 background GC: 0.46021305427243947 target length: 1000 num

### AMLN

In [86]:
%%bash
bed_file="/home/hmummey/data/ATAC/annotated_peak_calling/C57_AMLNDiet_round2_filt_annot_fin.bed" #+ peaks, no chrY, no chrUn
output=$(echo $bed_file | sed 's/.bed/_bg/g')
echo $output
python /home/zes017/Spacing/Codes/generate_background_coordinates.py $bed_file $output

/home/hmummey/data/ATAC/annotated_peak_calling/C57_AMLNDiet_round2_filt_annot_fin_bg
reading genome mm10
fasta file does not exist: chr20
fasta file does not exist: chr21
fasta file does not exist: chr22
done reading genome
0 0
target GC: 0.3800549163179718 background GC: 0.3820771278930075 target length: 1000 numTargetPositions 1912 backgroundPositions 1912
0 0
target GC: 0.4144524058577189 background GC: 0.41463975355186583 target length: 1000 numTargetPositions 1912 backgroundPositions 1912
0 0
target GC: 0.4324071166928084 background GC: 0.4303937820421112 target length: 1000 numTargetPositions 1911 backgroundPositions 1911
0 0
target GC: 0.4472857142856909 background GC: 0.44361969793678624 target length: 1000 numTargetPositions 1911 backgroundPositions 1911
0 0
target GC: 0.460633176347438 background GC: 0.4564509274085174 target length: 1000 numTargetPositions 1911 backgroundPositions 1911
0 0
target GC: 0.47450863422289513 background GC: 0.4688759696608745 target length: 1000 n